# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.svm import SVC

# Load All Datasets

In [ ]:
# Main X and Y data
x_train = pd.read_csv('Data/x_train_gr_smpl.csv')
y_train = pd.read_csv('Data/y_train_smpl.csv')


print(x_train.shape, y_train.shape)

# 10-Fold Cross Validation Linear Classifier Function

In [ ]:
def linear_classifier(x_train, y_train):
    
    y_train = np.array(y_train).flatten()
    
    model = SVC()
    model.fit(x_train, y_train)

    scores = cross_val_score(model, x_train, y_train, cv=2)
    np.set_printoptions(precision=2)
    
    return print('Scores: ',scores, ' Mean Score: ', np.round(scores.mean(), 2))

In [ ]:
linear_classifier(x_train, y_train)

# 10-Fold Cross Validation on Training Data using a Neural Network Function

In [ ]:
def neural_network(hidden_layers, neurons, l_rate, epochs, momentum):
    x_train = pd.read_csv('Data/x_train_gr_smpl.csv')
    y_train = pd.read_csv('Data/y_train_smpl.csv')

    x_train = x_train/255
    
    acc_per_fold = []
    loss_per_fold = []
    
    
    kfold = KFold(n_splits=10, shuffle=True)
    fold_no = 1
    
    for train, test in kfold.split(x_train, y_train):
        # Neural Network Model
        model = Sequential()
        model.add(Dense(2304, input_dim = 2304, activation='relu'))

        for i in range(hidden_layers):
            model.add(Dense(units=neurons, activation='relu'))

        model.add(Dense(units=10, activation='softmax'))

        opt = SGD(learning_rate=l_rate, momentum=momentum) 
        model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        
        print('-----------------------------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')
        
        x_train = x_train.reindex(index=range(len(x_train)))
        y_train = y_train.reindex(index=range(len(x_train)))
        
        model.fit(x_train.iloc[train], y_train.iloc[train], epochs=epochs)
        
        scores = model.evaluate(x_train.iloc[test], y_train.iloc[test])
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

        fold_no = fold_no + 1
        
        y_pred = np.argmax(model.predict(x_train.iloc[test], verbose=0), axis=-1)
        
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        
        # Analyzing the results
        # Confusion Matrix
        cm = metrics.confusion_matrix(y_train.iloc[test], y_pred)
        print('Confusion Matrix:')
        print(cm)
        print('\n')

        # True Positive
        TP = np.diag(cm)
        # False Positive
        FP = cm.sum(axis=0) - TP
        # False Negative
        FN = cm.sum(axis=1) - TP
        # True Negative
        TN = cm.sum() - (FP+FN+TP)

        #True Positive Rate
        TPR = TP/(TP+FN)
        TPR_mean = np.round(TPR.mean(), 2)
        #False Positive Rate
        FPR = FP/(FP+TN)
        FPR_mean = np.round(FPR.mean(), 2)
        #Precision
        Precision = TP/(TP+FP)
        Precision_mean = np.round(Precision.mean(), 2)
        #Recall
        Recall = TP/(TP+FN)
        Recall_mean = np.round(Recall.mean(), 2)
        #F1 Measure
        F1 = 2 * (Precision * Recall) / (Precision + Recall)
        F1_mean = np.round(F1.mean(), 2)

        np.set_printoptions(precision=2)

        print('TP rate: ', TPR)
        print('Mean TP rate: ', TPR_mean)
        print('FP rate: ', FPR)
        print('Mean FP rate: ', FPR_mean)
        print('Precision: ', Precision)
        print('Mean Precision: ', Precision_mean)
        print('Recall: ', Recall)
        print('Mean Recall: ', Recall_mean)
        print('F Measure: ', F1)
        print('Mean F Measure: ', F1_mean)
        print('\n')

        print('Classification Report:')
        print(metrics.classification_report(y_train.iloc[test], y_pred))

        # Visualizing the results
        # Heatmap
        print('\n')
        print('\n')
        print('Visualization of Results')
        plt.figure(figsize=(11,7))
        sns.heatmap(cm, annot=True, fmt=' .0f', square=True, cmap='tab20b')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Heatmap of Confusion Matrix')


        plt.figure(figsize=(8,4))
        plt.scatter(Precision, Recall, c='purple')
        plt.xlabel('Precsion')
        plt.ylabel('Recall')
        plt.title('Precision vs Recall Scatter Chart')
        plt.show()
        
        print('\n')
        print('-----------------------------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------------------------')
        print('\n')

    
    print('-----------------------------------------------------------------------------------------------------------')
    print('Score per fold: ')
    for i in range(0, len(acc_per_fold)):
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print('-----------------------------------------------------------------------------------------------------------')
    print('Average scores for all folds: ')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')
    print('-----------------------------------------------------------------------------------------------------------')

# Varying the Neural Network Parameters

# Hidden Layers (Other Parameterss remain Constant)

# 1 Hidden Layer

In [ ]:
neural_network(hidden_layers=1, neurons=100, l_rate=0.01, epochs=5, momentum=0.9)

# 2 Hidden Layers

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=5, momentum=0.9)

# 4 Hidden Layers

In [ ]:
neural_network(hidden_layers=4, neurons=100, l_rate=0.01, epochs=5, momentum=0.9)

# Neurons in Hidden Layers (Other Parameters remain Constant)

# 50 Neurons

In [ ]:
neural_network(hidden_layers=2, neurons=50, l_rate=0.01, epochs=5, momentum=0.9)

# 100 Neurons

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=5, momentum=0.9)

# 150 Neurons

In [ ]:
neural_network(hidden_layers=2, neurons=150, l_rate=0.01, epochs=5, momentum=0.9)       

# Learning Rate (Other Parameters remian Constant)

# Learning Rate = 0.005

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.005, epochs=5, momentum=0.9)

# Learning Rate = 0.01

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=5, momentum=0.9)

# Learning Rate = 0.02

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.02, epochs=5, momentum=0.9)

# Epochs (Other Parameters remain Constant)

# Epochs = 5

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=5, momentum=0.9)

# Epochs = 10

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=10, momentum=0.9)

# Momentum (Other Parameters remain Constant)

# Momentum = 0.5

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=5, momentum=0.5)

# Momentum = 0.7

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=5, momentum=0.7)

# Momentum = 0.9

In [ ]:
neural_network(hidden_layers=2, neurons=100, l_rate=0.01, epochs=5, momentum=0.9)